In [ ]:
%pip install langchain --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 11.8 MB/s eta 0:00:00


In [ ]:
import torch
import transformers
from transformers import NllbTokenizer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from time import time

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
fullDoc = "Ladies and gentlemen, esteemed delegates, As we gather here today under the auspices of the United Nations, it is incumbent upon us to recognize both the gravity of the challenges we face and the power of collective action to surmount them. In every corner of the globe, humanity grapples with pressing issues that transcend borders and ideologies. From the existential threat of climate change to the persistent scourge of poverty and inequality, our world stands at a crossroads. Yet, in the face of these daunting challenges, we must not succumb to despair but rather redouble our efforts to forge a brighter future for all. The principles enshrined in"


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
test_sentence = 'This is a test sentence. I am trying to split text...It works well. Now tested ellipses'
print(test_sentence.split('.'))

['This is a test sentence', ' I am trying to split text', '', '', 'It works well', ' Now tested ellipses']


In [ ]:
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size = 300,
#     length_function = len,
#     is_separator_regex = False,
# )
class text_splitter():
    def __init__(self, strategy='sentence'):
        self.strategy = strategy

    def split(self, document=''):
        if(self.strategy == 'sentence'):
            return document.split('.')

# texts = text_splitter.create_documents([fullDoc])
splitter = text_splitter()
texts = splitter.split(fullDoc)
for i, text in enumerate(texts):
    print(f'doc: #{i}', text)

doc: #0 Ladies and gentlemen, esteemed delegates, As we gather here today under the auspices of the United Nations, it is incumbent upon us to recognize both the gravity of the challenges we face and the power of collective action to surmount them
doc: #1  In every corner of the globe, humanity grapples with pressing issues that transcend borders and ideologies
doc: #2  From the existential threat of climate change to the persistent scourge of poverty and inequality, our world stands at a crossroads
doc: #3  Yet, in the face of these daunting challenges, we must not succumb to despair but rather redouble our efforts to forge a brighter future for all
doc: #4  The principles enshrined in


In [ ]:
tTimes = []
for text in texts:
    inputs = tokenizer(text, return_tensors="pt")
    init_time = time()
    translated_tokens = model.generate(
        **inputs, forced_bos_token_id=tokenizer.lang_code_to_id["fra_Latn"], max_length=100
    )
    translation = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
    end_time = time()
    process_time = end_time-init_time
    print(f'Time taken for translation = {process_time}s')
    tTimes.append(process_time/len(text.split(' ')))
    # print(f'Input article:\n{article}')
    # print(f'Translation:\n{translation}')
print(tTimes)
print(sum(tTimes)/len(tTimes))

Time taken for translation = 16.92689323425293s
Time taken for translation = 9.3235445022583s
Time taken for translation = 10.632060289382935s
Time taken for translation = 11.040708303451538s
Time taken for translation = 2.0638208389282227s
[0.4128510544939739, 0.5484437942504883, 0.4832754676992243, 0.40891512235005695, 0.4127641677856445]
0.45324992131587755
